<a href="https://colab.research.google.com/github/nikeyes/GenAI-experiments/blob/main/GenAI-experiments/LLM_with_your_own_pdfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

In [4]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [5]:

!wget https://www.adevinta.com/app/uploads/2023/04/Adevinta-AR2022.pdf

!mkdir pdfs
!cp *pdf '/content/pdfs'


--2023-05-16 09:29:45--  https://www.adevinta.com/app/uploads/2023/04/Adevinta-AR2022.pdf
Resolving www.adevinta.com (www.adevinta.com)... 99.84.208.41, 99.84.208.87, 99.84.208.25, ...
Connecting to www.adevinta.com (www.adevinta.com)|99.84.208.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10391355 (9.9M) [application/pdf]
Saving to: ‘Adevinta-AR2022.pdf’

Adevinta-AR2022.pdf 100%[===================>]   9.91M  17.9MB/s    in 0.6s    

2023-05-16 09:29:46 (17.9 MB/s) - ‘Adevinta-AR2022.pdf’ saved [10391355/10391355]



In [6]:
# connect your Google Drive
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
import os
pdf_folder_path = '/content/pdfs'
os.listdir(pdf_folder_path)

['Adevinta-AR2022.pdf']

In [7]:
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)).from_loaders(loaders)
     

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [17]:
import os
import requests
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('config.ini')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = parser.get('HuggingFace','ACCESS_TOKEN')

In [18]:
from langchain import HuggingFaceHub
#llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":512})
# llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":512})
# llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0, "max_length":512})
#llm=HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-783M", model_kwargs={"temperature":0, "max_length":512})

llm2=HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large", model_kwargs={"temperature":0, "max_length":512})

In [20]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm2, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [23]:
chain.run('List the Adevinta’s shareholders and their percent of shares outstanding')

ValueError: ignored

In [21]:
chain.run('How many tonnes of carbon emissions were potentially saved as a result of the transactions conducted on our platforms?')

"I don't know."

In [22]:
chain.run('List the names of Adevinta ASA’s Board of Directors and the number of their shares held in Adevinta')

ValueError: ignored

In [ ]:
ques = input('Your question: ')
chain.run(ques)